In [8]:
from bs4 import BeautifulSoup
import pandas as pd
import csv
import re

file_path = "/home/guillaume/Documents/Github/diag360/data/data_media/raw/medias_locaux.txt"

def extraire_donnees_media(chemin_fichier):
        with open(chemin_fichier, 'r', encoding='utf-8') as f:
            contenu = f.read()

        # 1. On récupère d'abord tout le texte entre les balises <a>...</a>
        # Le format est : <a href="#">Texte (Ville)</a>
        balises = re.findall(r'<a href="#">(.*?)</a>', contenu)

        data = []

        for item in balises:
            # 2. On sépare le nom de la ville
            # On cherche la DERNIÈRE parenthèse de la chaîne
            # (.*) -> Nom du média
            # \s -> espace
            # \(([^)]+)\)$ -> Contenu de la dernière parenthèse à la fin de la chaîne
            match = re.search(r'(.*)\s\(([^)]+)\)$', item)

            if match:
                nom_media = match.group(1).strip()
                ville = match.group(2).strip()
                data.append([nom_media, ville])
            else:
                # Cas de secours si le format est différent
                data.append([item, "Inconnue"])

        # 3. Création du DataFrame
        df = pd.DataFrame(data, columns=['Nom_media', 'Ville'])
        return df

    # Exécution
try:
    df_medias = extraire_donnees_media(str(file_path))
    print("Extraction réussie :")
    print(df_medias.head())

    # Optionnel : Sauvegarder en CSV
    df_medias.to_csv("medias_extraits.csv", index=False)
except Exception as e:
    print(f"Erreur lors de la lecture du fichier : {e}")

df_medias.head()

Extraction réussie :
        Nom_media                   Ville
0   Zones D'Ondes                    Caen
1          Tsf 98  Hérouville-Saint-Clair
2  Tendance Ouest                    Caen
3    Sport à Caen                    Caen
4       Radio VFM                    Caen


,Nom_media,Ville
0,Zones D'Ondes,Caen
1,Tsf 98,Hérouville-Saint-Clair
2,Tendance Ouest,Caen
3,Sport à Caen,Caen
4,Radio VFM,Caen


In [9]:
df_medias.head(70)

,Nom_media,Ville
0,Zones D'Ondes,Caen
1,Tsf 98,Hérouville-Saint-Clair
2,Tendance Ouest,Caen
3,Sport à Caen,Caen
4,Radio VFM,Caen
...,...,...
65,Ouest-France,Loudéac
66,Ouest-France,Guingamp
67,Ouest-France,Saint-Brieuc
68,Ouest-France,Lamballe-Armor


In [75]:
df_medias.to_csv("../data/processed/media.csv", index=False, sep=";")

In [10]:
import os
import sys
from pathlib import Path
import requests

def download_file(url: str, extract_to: str = '.', filename: str = None) -> None : 
    """
    Télécharge un fichier depuis une URL et l'enregistre localement.

    Le fichier est téléchargé uniquement s'il n'existe pas déjà
    dans le répertoire de destination.

    Parameters
    ----------
    url : str
        URL du fichier à télécharger.
    extract_to : str, optional
        Répertoire de destination du fichier (par défaut : répertoire courant).
    filename : str
        Nom du fichier local (avec extension).

    Raises
    ------
    requests.exceptions.RequestException
        En cas d'erreur réseau lors du téléchargement.
    """

    if not os.path.exists(extract_to):
        os.makedirs(extract_to, exist_ok=True)
        print(f"Dossier créé : {extract_to}")

    filename = os.path.join(extract_to, filename)

    if not os.path.exists(filename):
        response = requests.get(url)
        response.raise_for_status()
        print(f"Téléchargement du fichier : {filename}")

        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"Fichier téléchargé avec succès : {filename}")

def float_to_codepostal(df: pd.DataFrame, col: str) -> pd.DataFrame:
    """
    Convertit une colonne contenant des codes postaux numériques en format chaîne à 5 caractères.

    Cette fonction est destinée aux cas où les codes postaux ont été lus comme
    des nombres flottants (ex. `1400.0`) et doivent être restaurés en chaînes
    avec zéros initiaux (ex. `01400`).

    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame contenant la colonne à transformer.
    col : str
        Nom de la colonne contenant les codes postaux.

    Returns
    -------
    pandas.DataFrame
        DataFrame avec la colonne des codes postaux convertie en chaînes
        de longueur 5.

    Notes
    -----
    - La fonction modifie le DataFrame en place et le retourne.
    - Les valeurs manquantes sont converties en chaînes `'nan'`
      si elles ne sont pas nettoyées en amont.
    """

    df[col] = (
        df[col]
        .astype(str)
        .str.replace(".0", "", regex=False)
        .str.zfill(5)
    )
    return df

def create_dataframe_communes(dir_path):
    com_url = (
        "https://www.data.gouv.fr/api/1/datasets/r/f5df602b-3800-44d7-b2df-fa40a0350325"
    )
    download_file(com_url, extract_to=dir_path, filename="communes_france_2025.csv")
    df_com = pd.read_csv(dir_path / "communes_france_2025.csv")
    df_com = float_to_codepostal(df_com, "code_postal")
    return df_com

df_com = create_dataframe_communes(Path("../data/raw/"))

/tmp/ipykernel_24023/1996302946.py:84: DtypeWarning: Columns (1,12,14,16,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_com = pd.read_csv(dir_path / "communes_france_2025.csv")


In [11]:
df_com.head()

,Unnamed: 0,code_insee,nom_standard,nom_sans_pronom,nom_a,nom_de,nom_sans_accent,nom_standard_majuscule,typecom,typecom_texte,...,longitude_mairie,latitude_centre,longitude_centre,grille_densite,grille_densite_texte,niveau_equipements_services,niveau_equipements_services_texte,gentile,url_wikipedia,url_villedereve
0,0,01001,L'Abergement-Clémenciat,Abergement-Clémenciat,à Abergement-Clémenciat,de l'Abergement-Clémenciat,l-abergement-clemenciat,L'ABERGEMENT-CLÉMENCIAT,COM,commune,...,4.921,46.153,4.926,6,Rural à habitat dispersé,0.0,communes non pôle,NaN,https://fr.wikipedia.org/wiki/fr:L'Abergement-...,https://villedereve.fr/ville/01001-l-abergemen...
1,1,01002,L'Abergement-de-Varey,Abergement-de-Varey,à Abergement-de-Varey,de l'Abergement-de-Varey,l-abergement-de-varey,L'ABERGEMENT-DE-VAREY,COM,commune,...,5.423,46.009,5.428,6,Rural à habitat dispersé,0.0,communes non pôle,"Abergementais, Abergementaises",https://fr.wikipedia.org/wiki/fr:L'Abergement-...,https://villedereve.fr/ville/01002-l-abergemen...
2,2,01004,Ambérieu-en-Bugey,Ambérieu-en-Bugey,à Ambérieu-en-Bugey,d'Ambérieu-en-Bugey,amberieu-en-bugey,AMBÉRIEU-EN-BUGEY,COM,commune,...,5.360,45.961,5.373,2,Centres urbains intermédiaires,3.0,centres structurants d'équipements et de services,"Ambarrois, Ambarroises",https://fr.wikipedia.org/wiki/fr:Ambérieu-en-B...,https://villedereve.fr/ville/01004-amberieu-en...
3,3,01005,Ambérieux-en-Dombes,Ambérieux-en-Dombes,à Ambérieux-en-Dombes,d'Ambérieux-en-Dombes,amberieux-en-dombes,AMBÉRIEUX-EN-DOMBES,COM,commune,...,4.903,45.996,4.912,5,Bourgs ruraux,1.0,centres locaux d'équipements et de services,Ambarrois,https://fr.wikipedia.org/wiki/fr:Ambérieux-en-...,https://villedereve.fr/ville/01005-amberieux-e...
4,4,01006,Ambléon,Ambléon,à Ambléon,d'Ambléon,ambleon,AMBLÉON,COM,commune,...,5.601,45.750,5.594,6,Rural à habitat dispersé,0.0,communes non pôle,Ambléonais,https://fr.wikipedia.org/wiki/fr:Ambléon,https://villedereve.fr/ville/01006-ambleon


In [12]:
df_com.columns

Index(['Unnamed: 0', 'code_insee', 'nom_standard', 'nom_sans_pronom', 'nom_a',
       'nom_de', 'nom_sans_accent', 'nom_standard_majuscule', 'typecom',
       'typecom_texte', 'reg_code', 'reg_nom', 'dep_code', 'dep_nom',
       'canton_code', 'canton_nom', 'epci_code', 'epci_nom', 'academie_code',
       'academie_nom', 'code_postal', 'codes_postaux', 'zone_emploi',
       'code_insee_centre_zone_emploi', 'code_unite_urbaine',
       'nom_unite_urbaine', 'taille_unite_urbaine',
       'type_commune_unite_urbaine', 'statut_commune_unite_urbaine',
       'population', 'superficie_hectare', 'superficie_km2', 'densite',
       'altitude_moyenne', 'altitude_minimale', 'altitude_maximale',
       'latitude_mairie', 'longitude_mairie', 'latitude_centre',
       'longitude_centre', 'grille_densite', 'grille_densite_texte',
       'niveau_equipements_services', 'niveau_equipements_services_texte',
       'gentile', 'url_wikipedia', 'url_villedereve'],
      dtype='object')

In [13]:
import duckdb

In [15]:
query = """
SELECT 
    code_insee,
    nom_standard,
    dep_code,
    epci_code,
    epci_nom,
    df_medias.Nom_media AS nom_media  
FROM df_com  
INNER JOIN df_medias
ON df_com.nom_standard = df_medias.Ville
"""

df_result = duckdb.query(query).to_df()
df_result.shape


(2708, 6)

In [16]:
df_result[df_result["d"] == "14"]

KeyError: 'd'

## Suppression des doublons de villes

In [17]:
dup_com = (
    df_com
    .groupby("nom_standard")
    .size()
    .reset_index(name="n_com")
    .query("n_com > 1")
)

dup_com

,nom_standard,n_com
2,Abancourt,2
34,Aboncourt,2
45,Abzac,2
63,Achères,2
143,Aiglun,2
...,...,...
32581,Étaules,2
32590,Éterpigny,2
32636,Étréchy,3
32641,Étrépilly,2


In [18]:
dup_medias = (
    df_medias
    .groupby("Ville")
    .size()
    .reset_index(name="n_medias")
    .query("n_medias > 1")
)
dup_medias

,Ville,n_medias
2,Agen,7
6,Aire-sur-la-Lys,2
8,Aix-en-Provence,5
10,Ajaccio,6
11,Albertville,4
...,...,...
949,Ébreuil,2
950,Épernay,3
951,Épinal,7
954,Évreux,6


In [19]:
villes_ambigues = (
    dup_com
    .merge(dup_medias, left_on="nom_standard", right_on="Ville", how="inner")
)

villes_ambigues

,nom_standard,n_com,Ville,n_medias
0,Bailleul,3,Bailleul,2
1,Blanquefort,2,Blanquefort,3
2,Castres,2,Castres,2
3,Chaumont,5,Chaumont,6
4,Clamecy,2,Clamecy,3
5,Falaise,2,Falaise,2
6,Flers,3,Flers,2
7,Fontaine,3,Fontaine,2
8,La Rochelle,2,La Rochelle,8
9,Langon,2,Langon,3


In [27]:
RULES_CONFIG = {
        "Bailleul": "59",
        "Castres": "81",
        "Chaumont": "52",
        "Clamecy": "58",
        "Falaise": "14",
        "Flers": "61",
        "Fontaine": "38",
        "La Rochelle": "17",
        "Langon": "33",
        "Marmagne": "71",
        "Montreuil": "93",
        "Moulins": "03",
        "Olivet": "45",
        "Prades": "66",
        "Rochefort": "17",
        "Saint-Claude": "39",
        "Saint-Nazaire": "44",
        "Saint-Omer": "62",
        "Saint-Raphaël": "83",
        "Ussel": "19",
        "Verdun": "55",
        "Vernon": "27",

        # Cas complexes avec conditions multiples
        "Blanquefort": lambda r: r["dep_code"] == '33' and r["nom_media"] == "R.I.G",

        "Valence": lambda r: (
            (r["dep_code"] == '82' and r["nom_media"] in ["VFM", "La Dépêche du Midi"]) or
            (r["dep_code"] == '26' and r["nom_media"] not in ["VFM", "La Dépêche du Midi"])
        )
    }

def filter_logic(row):
    ville = row["nom_standard"]

        # Si la ville n'est pas dans le dictionnaire, on garde la ligne par défaut
    if ville not in RULES_CONFIG:
        return True

    regle = RULES_CONFIG[ville]

        # Si la règle est une fonction (cas complexes)
    if callable(regle):
        return regle(row)

        # Sinon, c'est une règle simple de département (comparaison directe)
    return row["dep_code"] == regle

    # Application du filtre en une seule ligne
df_temp = df_result[df_result.apply(filter_logic, axis=1)].copy()

df_temp.drop_duplicates(inplace=True)
print(f"Après filtrage, df_temp.shape: {df_temp.shape}")

Après filtrage, df_temp.shape: (2506, 6)


In [29]:
#différence des villes entre df_médias et df_result_final
set_villes_medias = set(df_medias["Ville"].unique())
set_villes_result = set(df_temp["nom_standard"].unique())
set_villes_diff = set_villes_medias - set_villes_result
set_villes_diff

{'Bourg Les Valence',
 'Charleville-Mézieres',
 'Cherbourg',
 'Cherbourg-En-Cotentin',
 'Château du Loir',
 'Cierp Gaud',
 'Digne les Bains',
 'Echouboulains',
 'Inconnue',
 'SAINT-AIGNAN DE GRAND LIEU',
 'Saint-Quentin-en-Yvelines',
 'Sanary',
 'St Philbert de Grand-Lieu',
 'Vaux-Sur-Mer',
 'la Seyne'}

In [ ]:
ville_mapping = {
    "Bourg Les Valence": "Bourg-lès-Valence",
    "Charleville-Mézieres": "Charleville-Mézières",
    "Cherbourg": "Cherbourg-en-Cotentin",
    "Cherbourg-En-Cotentin": "Cherbourg-en-Cotentin",
    "Château du Loir": "Montval-sur-Loir",
    "Cierp Gaud": "Cierp-Gaud",
    "Digne les Bains": "Digne-les-Bains",
    "Echouboulains": "Échouboulains",
    'Inconnue': "Château-Chinon (Ville)",
    "SAINT-AIGNAN DE GRAND LIEU": "Saint-Aignan-Grandlieu",
    "Saint-Quentin-en-Yvelines": "Montigny-le-Bretonneux",
    "Sanary": "Sanary-sur-Mer",
    "St Philbert de Grand-Lieu": "Saint-Philbert-de-Grand-Lieu",
    "Vaux-Sur-Mer": "Vaux-sur-Mer",
    "la Seyne": "La Seyne-sur-Mer",
}

df_medias["Ville"] = df_medias["Ville"].replace(ville_mapping)

NameError: name 'df_medias' is not defined

In [ ]:
nom_mapping = {
    'Mistral Social Club': 'Salon-de-Provence',
    'Mon Pays': 'Toulouse',
    'Tamtam': 'Bezons',
}

df_medias["Ville"] = df_medias["Nom"].replace(nom_mapping)

In [32]:
df_medias.loc[df_medias["Ville"] ==  "Inconnue"
               , "Nom_media"]

1588    Frequence Morvan Force 5 (Château-Chinon (Ville))
Name: Nom_media, dtype: object

In [33]:
df_com.loc[df_com["nom_standard"] == "Château-Chinon (Ville)"]

,Unnamed: 0,code_insee,nom_standard,nom_sans_pronom,nom_a,nom_de,nom_sans_accent,nom_standard_majuscule,typecom,typecom_texte,...,longitude_mairie,latitude_centre,longitude_centre,grille_densite,grille_densite_texte,niveau_equipements_services,niveau_equipements_services_texte,gentile,url_wikipedia,url_villedereve
21533,21533,58062,Château-Chinon (Ville),Château-Chinon (Ville),à Château-Chinon (Ville),de Château-Chinon (Ville),chateau-chinon-(ville),CHÂTEAU-CHINON (VILLE),COM,commune,...,3.932,47.063,3.927,5,Bourgs ruraux,2.0,centres intermédiaires d'équipements et de ser...,Château-Chinonais,https://fr.wikipedia.org/wiki/fr:Château-Chino...,https://villedereve.fr/ville/58062-chateau-chi...


In [78]:
df_medias[df_medias["Ville"].isin(set_villes_diff)].shape

(87, 2)

In [49]:
# Finaliser le DataFrame
df_result_final = df_temp.copy()

In [66]:
#ligne à supprimer
import requests
url_media_non_independants = "https://raw.githubusercontent.com/mdiplo/Medias_francais/refs/heads/master/medias.tsv"
df = pd.read_csv(url_media_non_independants, sep = "\t")
df

,Nom,Type,Periodicite,Echelle,Prix,Disparu
0,6ter,Télévision,NaN,NaN,Gratuit,NaN
1,20 Minutes,Site,NaN,NaN,Gratuit,NaN
2,ARC Info,Presse (généraliste politique économique),NaN,Suisse,NaN,NaN
3,Arte,Télévision,NaN,Europe,Gratuit,NaN
4,Aujourd’hui en France,Presse (généraliste politique économique),Quotidien,National,Payant,NaN
...,...,...,...,...,...,...
213,Vaucluse Matin,Presse (généraliste politique économique),Quotidien,Régional,Payant,NaN
214,Voici,Presse (généraliste politique économique),Hebdomadaire,National,Payant,NaN
215,Vosges Matin,Presse (généraliste politique économique),Quotidien,Régional,Payant,NaN
216,VSD,Presse (généraliste politique économique),Mensuel,National,Payant,NaN


In [67]:
#on retire de df_result_final les médias présents dans df
df_final = df_result_final[~df_result_final["media_nom"].isin(df["Nom"])]
df_final.shape

(2223, 6)

In [68]:
df_final.head(50)

,code_insee,nom_standard,dep_code,epci_code,epci_nom,media_nom
0,01004,Ambérieu-en-Bugey,01,240100883,CC de la Plaine de l'Ain,Le Journal du Bugey
2,01033,Valserhône,01,240100891,CC du Pays Bellegardien (CCPB),Exil Sorgia FM
3,01053,Bourg-en-Bresse,01,200071751,CA du Bassin de Bourg-en-Bresse,France 3 Rhone Alpes Auvergne
4,01159,Feillens,01,200071371,CC Bresse et Saône,L'Aindépendant
5,01160,Ferney-Voltaire,01,240100750,CA du Pays de Gex,Zones
6,01262,Montluel,01,240100610,CC de la Côtière à Montluel,FC Radio
8,01328,Romans,01,200069193,CC de la Dombes,Zig-Zag
9,01344,Saint-Denis-lès-Bourg,01,200071751,CA du Bassin de Bourg-en-Bresse,Eco de l'Ain
11,02168,Château-Thierry,02,200072031,CA de la Région de Château-Thierry,R2M La Radio Plus
13,02361,Guise,02,200071983,CC Thiérache Sambre et Oise,L'Aisne nouvelle


In [69]:
query_by_dept = """ 
SELECT 
    dep_code as dept,
    count(media_nom) AS n_medias
FROM df_final
GROUP BY dep_code
ORDER BY dep_code
"""

nb_medias_par_dept = duckdb.query(query_by_dept).to_df()
nb_medias_par_dept

,dept,n_medias
0,01,19
1,02,14
2,03,15
3,04,12
4,05,11
...,...,...
94,94,5
95,95,10
96,971,1
97,972,1


In [70]:
query_by_epci = """ 
SELECT
    dep_code as dept,
    epci_code,
    count(media_nom) AS n_medias
FROM df_final
GROUP BY dep_code, epci_code
ORDER BY dep_code, epci_code
"""

nb_medias_par_epci = duckdb.query(query_by_epci).to_df()
nb_medias_par_epci

,dept,epci_code,n_medias
0,01,200069193,1
1,01,200071371,1
2,01,200071751,11
3,01,240100610,2
4,01,240100750,1
...,...,...,...
597,95,249500513,1
598,971,249710047,1
599,972,200041788,1
600,974,249740077,1
